# 2.3 The Tidy and Data Cube Forms

In the previous section, we saw how to repeat an analysis for every slice of a variable, by splitting the `DataFrame` on that variable using `.groupby()`:

In [ ]:
%matplotlib inline
import pandas as pd
df = pd.read_csv("/data301/data/titanic.csv")

df.groupby("pclass").survived.sum()

We can also repeat an analysis for every combination of values of multiple variables:

In [ ]:
survival_counts = df.groupby(["pclass", "sex"]).survived.sum()
survival_counts

Note that this is a `Series` with a two-level index, not a `DataFrame` with three columns. If we wanted `pclass` and `sex` to instead be columns in a `DataFrame`, we could reset the index:

In [ ]:
survival_counts.reset_index()

Either way, there is exactly one row per unique combination of `pclass` and `sex`. When each row represents a distinct combination of values of variables, it is said to be in **tidy form**. Many libraries expect data to be in this form, but tidy form is sometimes inconvenient to read. For example, in the above data frame, it is not easy to compare the number of females who survived across classes 1, 2, and 3.

## Data Cube Form

It would be more convenient if the data were arranged so that the values of `pclass` are along one dimension and the values of `sex` are along another. We can achieve this by **pivoting** the data. The easiest way to do this in `pandas` is using the `.unstack()` method, which turns the innermost level of the index into columns.

In [ ]:
survival_counts_cube = survival_counts.unstack()
survival_counts_cube

When there is one variable along each dimension, as in the `DataFrame` above, the data is said to be in **data cube form**. Data cube form makes it easy to compare the levels of one variable, fixing the value of the other variable. For example, we can compare the number of female survivors across the three passenger classes by simply looking at the first column.

Data cube form also happens to play nicely with bar charts in `pandas`. When `.plot.bar()` is called on a `pandas` `DataFrame`, one set of bars will be created for each column. So when we call `.plot.bar()` on the above `DataFrame`, we will get one set of bars for females and another set of bars for males.

In [ ]:
survival_counts_cube.plot.bar()

Notice that the $x$-axis is still the index of this `DataFrame`, which is `pclass`.

Although data cube form is convenient to read, it is not the canonical form the many libraries expect. However, given a `DataFrame` in data cube form, we can convert it to tidy form by using `.stack()` (which is the opposite of `.unstack()` above).

In [ ]:
survival_counts_cube.stack()

## Roll-ups

In the above analysis, we calculated the number of survivors by `pclass` and `sex`. What if we wanted to use the results of this analysis to calculate the number of survivors by just `pclass`? To do this, we have to **roll-up** the variable `sex`.

The term _roll-up_ makes sense if the data is in data cube format. Right now, the data has two dimensions, `pclass` and `sex`. We want to reduce the data to just one dimension: `pclass`. To do this, we have to aggregate, or _roll-up_, the values over the `sex` dimension. We choose an aggregation function, like `.sum()` or `.mean()` or `.max()`. Although we have so far only applied these aggregation functions to `Series` (i.e., individual variables), they can also be applied to `DataFrame`s. When applied to a `DataFrame`, they take an additional keyword argument, `axis=`, that specifies the dimension to aggregate over.

- `axis=0` means aggregate _over_ the rows (i.e., dimension 0), returning one number per column
- `axis=1` means aggregate _over_ the columns (i.e., dimension 1), returning one number per row

In this case, we need to add up the number of male and female survivors, so we use the `.sum()` aggregation function.

In [ ]:
survival_counts_cube.sum(axis=1)

Roll-ups can also be calculated when the data is in tidy form, although it is not as natural. We simply need to `.groupby()` the dimension(s) we want to keep and aggregate over the rest. However, 

`survival_counts.groupby("pclass").sum()`

won't work here. (Try it!) That's because `pclass` is not a column in `survival_counts`; it's in the index! To group by it, we either have to reset the index so that it *is* a column:

In [ ]:
survival_counts.reset_index().groupby("pclass").survived.sum()

or specify to `.groupby()` to look for it in the index, using the `level=` keyword argument:

In [ ]:
survival_counts.groupby(level="pclass").sum()

# Exercises

Exercises 1-3 all deal with the Tips dataset (`/data301/data/tips.csv`).

**Exercise 1.** Calculate the average total bill by day, time, and table size. Show the `DataFrame` in data cube format. Because there are three variables, and a `DataFrame` is two-dimensional, you will need to have two variables along one dimension and one variable along the other.

Try different ways of arranging the data cube. For example, you could have:

- each row represent a day and time combination and each column represent a size.
- each row represent a day and each column represent a time and size combination.

In [ ]:
# TYPE YOUR CODE HERE.

**Exercise 2.** Using the data cube that you created in Exercise 1, roll-up the `size` variable by calculating the average total bill over the different table sizes. (_Hint:_ This can be easy or hard, depending on the data cube representation you chose in Exercise 1.)

Compare these results to the results you get when you calculate the average total bill by day and time from the original `DataFrame`. Why do they differ?

In [ ]:
# TYPE YOUR CODE HERE.

**Exercise 3.** Make a bar chart showing the average total bill by table size, day, and time. (You will have to decide which variable(s) to represent on the $x$-axis and which variable(s) to represent using different colored bars.)

In [ ]:
# TYPE YOUR CODE HERE.